In [3]:
"""
필요한 모듈 및 프레임워크를 잘 생각하고 넣자
"""
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from keras.preprocessing.image import img_to_array
import tensorflow as tf
import numpy as np
import pandas as pd 
import os, re

In [6]:
"""
전처리 후 npy파일로 저장하기위한 코드
일단 사이즈는 강제로 256,256으로 변환시키는 방식
각 이미지를 일차원 배열로 생성
사용시에는 인덱스에 맞게 꺼내서 사용할 예정
"""

current_path = os.path.abspath('')
input_img_path = os.path.join(current_path,"input_img")
target_img_path = os.path.join(current_path,"target_img")
input_files = os.listdir(input_img_path)
target_files = os.listdir(target_img_path)
datasets = []
i = 0
test_data = np.array([])
for input_file, target_file in zip(input_files, target_files):
    dataset= []
    plus_input = []
    mul_input = []
    input_pro = []
    name = str(input_file)
    #레이놀즈 수 파일 이름에서 추출
    R = float(name[5]+"."+name[6]+name[7])*10**int(name[8])/(1.3*(10**6))
    #인풋 이미지 파일 경로로 들어가서 인풋 이미지 전처리 과정
    os.chdir(input_img_path)
    input_img = Image.open(input_file)
    #이미지 리사이징
    input_img = input_img.resize((256, 256))
    #이미지 nunpy배열로 변환
    input_img_tensor = img_to_array(input_img)
    input_img_tensor = input_img_tensor.reshape(196608).astype('float64')/ 255
    #타겟 이미지 파일 경로로 들어가서 전처리 과정
    os.chdir(target_img_path)
    target_img = Image.open(target_file)
    target_img = target_img.resize((256,256))
    #타겟 이미지는 1차원 배열로 생성
    target_img_tensor = img_to_array(target_img)
    target_img_tensor = target_img_tensor.reshape(196608).astype('float64')/ 255
    plus_input = (input_img_tensor +R)/2
    dataset = np.concatenate((input_img_tensor,plus_input), axis = 0)
    dataset = np.concatenate((dataset,target_img_tensor), axis = 0)
    datasets.insert(i,dataset)
    i += 1
    os.chdir('..')
datasets = np.array(datasets)
#[:,:196608]인풋 이미지,[:,196608:393216]레이놀즈수,[:,393216:]타겟 이미지
np.save("test.npy", datasets)
print(datasets.shape)

(1169, 589824)


In [7]:

dataset = np.load('test.npy', allow_pickle=True)
X = dataset[:,196608:393216]
Y = dataset[:,393216:]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state =1)
X_test, X_val,Y_test, Y_val = train_test_split(X_test,Y_test,test_size=0.5,random_state=1)
X_train = X_train.reshape(X_train.shape[0], 256,256,3)
X_test = X_test.reshape(X_test.shape[0], 256,256,3)
X_val = X_val.reshape(X_val.shape[0], 256,256,3)
Y_train = Y_train.reshape(Y_train.shape[0], 256,256,3)
Y_test = Y_test.reshape(Y_test.shape[0], 256,256,3)
Y_val = Y_val.reshape(Y_val.shape[0], 256,256,3)


In [8]:
"""
VGG16과 u-net의 조합을 해볼 생각이다.
"""
input_e = tf.keras.Input(shape=(256,256,3))

block1_conv1 = layers.Conv2D(64, 3, activation='relu', padding = 'same')(input_e)
block1_conv2 = layers.Conv2D(64, 3, activation='relu', padding = 'same')(block1_conv1)
block1_pool = layers.MaxPooling2D(2)(block1_conv2)
        
block2_conv1 = layers.Conv2D(128, 3, activation='relu', padding = 'same')(block1_pool)
block2_conv2 = layers.Conv2D(128, 3, activation='relu', padding = 'same')(block2_conv1)
block2_pool = layers.MaxPooling2D(2)(block2_conv2)

block3_conv1 = layers.Conv2D(256, 3, activation='relu', padding = 'same')(block2_pool)
block3_conv2 = layers.Conv2D(256, 3, activation='relu', padding = 'same')(block3_conv1)
block3_conv3 = layers.Conv2D(256, 3, activation='relu', padding = 'same')(block3_conv2)
block3_pool = layers.MaxPooling2D(2)(block3_conv3)

block4_conv1 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block3_pool)
block4_conv2 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block4_conv1)
block4_conv3 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block4_conv2)
block4_pool = layers.MaxPooling2D(2)(block4_conv3)

block5_conv1 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block4_pool)
block5_conv2 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block5_conv1)
block5_conv3 = layers.Conv2D(512, 3, activation='relu', padding = 'same')(block5_conv2)
block5_pool = layers.MaxPooling2D(2)(block5_conv3)
"""
여기까지가 VGG16을 구현한 장소
"""
block6_convt1 = layers.Conv2DTranspose(512, 3, activation='relu', padding='same')(block5_pool)
block6_upsamp = layers.UpSampling2D(2)(block6_convt1)
block6_skipcon = layers.Concatenate(axis=3)([block4_pool, block6_upsamp])
block6_convt2 = layers.Conv2D(512, 3, activation = 'relu', padding='same')(block6_skipcon)

block7_convt1 = layers.Conv2DTranspose(256, 3, activation='relu', padding='same')(block6_convt2)
block7_upsamp = layers.UpSampling2D(2)(block7_convt1)
block7_skipcon = layers.Concatenate(axis=3)([block3_pool, block7_upsamp])
block7_convt2 = layers.Conv2D(256, 3, activation = 'relu', padding='same')(block7_skipcon)

block8_convt1 = layers.Conv2DTranspose(128, 3, activation='relu', padding='same')(block7_convt2)
block8_upsamp = layers.UpSampling2D(2)(block8_convt1)
block8_skipcon = layers.Concatenate(axis=3)([block2_pool, block8_upsamp])
block8_convt2 = layers.Conv2D(128, 3, activation = 'relu', padding='same')(block8_skipcon)

block9_convt1 = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(block8_convt2)
block9_upsamp = layers.UpSampling2D(2)(block9_convt1)
block9_skipcon = layers.Concatenate(axis=3)([block1_pool, block9_upsamp])
block9_convt2 = layers.Conv2D(64, 3, activation = 'relu', padding='same')(block9_skipcon)

block10_convt1 = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(block9_convt2)
block10_upsamp = layers.UpSampling2D(2)(block10_convt1)

output_d = layers.Conv2DTranspose(3, 3, activation='relu', padding='same')(block10_upsamp)

model = keras.Model(input_e, output_d, name = 'unet-VGG16')

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0012, decay = 0.013),loss='mse', metrics=['mae'])
history = model.fit(X_train,Y_train,epochs=1000, validation_data=(X_val, Y_val), batch_size=20)

#hist = model.fit(X_tr,Y_tr['M'],epochs=500, validation_data=(X_vid, Y_vid['M']), batch_size=32)

model.summary()
model.save('test_model_unet_VGG.h5')

Epoch 1/1000
47/47 [==============================] - 26s 419ms/step - loss: 186429.0938 - mae: 47.9448 - val_loss: 0.3393 - val_mae: 0.3921
Epoch 2/1000
47/47 [==============================] - 15s 318ms/step - loss: 0.3376 - mae: 0.3884 - val_loss: 0.3342 - val_mae: 0.3844
Epoch 3/1000
47/47 [==============================] - 15s 321ms/step - loss: 0.3309 - mae: 0.3775 - val_loss: 0.3308 - val_mae: 0.3757
Epoch 4/1000
47/47 [==============================] - 15s 326ms/step - loss: 0.3285 - mae: 0.3735 - val_loss: 0.3293 - val_mae: 0.3685
Epoch 5/1000
47/47 [==============================] - 16s 339ms/step - loss: 0.3274 - mae: 0.3713 - val_loss: 0.3281 - val_mae: 0.3678
Epoch 6/1000
47/47 [==============================] - 15s 326ms/step - loss: 0.3272 - mae: 0.3708 - val_loss: 0.3278 - val_mae: 0.3675
Epoch 7/1000
47/47 [==============================] - 15s 312ms/step - loss: 0.3268 - mae: 0.3700 - val_loss: 0.3278 - val_mae: 0.3678
Epoch 8/1000
47/47 [=============================

47/47 [==============================] - 14s 301ms/step - loss: 0.3070 - mae: 0.3308 - val_loss: 0.3103 - val_mae: 0.3329
Epoch 62/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3070 - mae: 0.3307 - val_loss: 0.3103 - val_mae: 0.3323
Epoch 63/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3070 - mae: 0.3308 - val_loss: 0.3103 - val_mae: 0.3323
Epoch 64/1000
47/47 [==============================] - 14s 303ms/step - loss: 0.3070 - mae: 0.3306 - val_loss: 0.3103 - val_mae: 0.3324
Epoch 65/1000
47/47 [==============================] - 15s 329ms/step - loss: 0.3069 - mae: 0.3304 - val_loss: 0.3103 - val_mae: 0.3322
Epoch 66/1000
47/47 [==============================] - 14s 305ms/step - loss: 0.3069 - mae: 0.3305 - val_loss: 0.3103 - val_mae: 0.3325
Epoch 67/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3070 - mae: 0.3307 - val_loss: 0.3103 - val_mae: 0.3321
Epoch 68/1000
47/47 [==============================] - 14s 301

47/47 [==============================] - 14s 301ms/step - loss: 0.3066 - mae: 0.3285 - val_loss: 0.3100 - val_mae: 0.3308
Epoch 122/1000
47/47 [==============================] - 14s 303ms/step - loss: 0.3067 - mae: 0.3287 - val_loss: 0.3100 - val_mae: 0.3304
Epoch 123/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3066 - mae: 0.3286 - val_loss: 0.3100 - val_mae: 0.3304
Epoch 124/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3067 - mae: 0.3287 - val_loss: 0.3100 - val_mae: 0.3306
Epoch 125/1000
47/47 [==============================] - 14s 303ms/step - loss: 0.3066 - mae: 0.3286 - val_loss: 0.3100 - val_mae: 0.3305
Epoch 126/1000
47/47 [==============================] - 14s 302ms/step - loss: 0.3066 - mae: 0.3285 - val_loss: 0.3100 - val_mae: 0.3303
Epoch 127/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3066 - mae: 0.3287 - val_loss: 0.3100 - val_mae: 0.3307
Epoch 128/1000
47/47 [==============================] - 

Epoch 181/1000
47/47 [==============================] - 14s 303ms/step - loss: 0.3065 - mae: 0.3276 - val_loss: 0.3099 - val_mae: 0.3295
Epoch 182/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3065 - mae: 0.3276 - val_loss: 0.3099 - val_mae: 0.3301
Epoch 183/1000
47/47 [==============================] - 14s 302ms/step - loss: 0.3065 - mae: 0.3278 - val_loss: 0.3099 - val_mae: 0.3296
Epoch 184/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3065 - mae: 0.3276 - val_loss: 0.3099 - val_mae: 0.3294
Epoch 185/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3065 - mae: 0.3276 - val_loss: 0.3099 - val_mae: 0.3295
Epoch 186/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3065 - mae: 0.3276 - val_loss: 0.3099 - val_mae: 0.3295
Epoch 187/1000
47/47 [==============================] - 14s 301ms/step - loss: 0.3065 - mae: 0.3277 - val_loss: 0.3100 - val_mae: 0.3306
Epoch 188/1000
47/47 [===================

KeyboardInterrupt: 

In [5]:
os.chdir('..')